In [13]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg, count, round, when, to_timestamp, hour
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# -----------------------------
# Configuración de Spark
# -----------------------------
spark = SparkSession.builder \
    .appName("Nettalco Big Data Processing") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Ruta de salida en Cloud Storage
output_path = "gs://nettalco-data-bd_grupo05/curated/"
# Rutas de los archivos CSV
detalle_path = "gs://nettalco-data-bd_grupo05/detalle-produccion-costura.csv"
cliente_path = "gs://nettalco-data-bd_grupo05/produccion-costura-cliente.csv"
linea_path = "gs://nettalco-data-bd_grupo05/produccion-costura-linea-cliente.csv"
segundas_path = "gs://nettalco-data-bd_grupo05/segundas-prendas.csv"

# Ruta de salida
output_path = "gs://nettalco-data-bd_grupo05/curated/"

# Cargar los datos
detalle_produccion = spark.read.csv(detalle_path, header=True, inferSchema=True)
produccion_costura_cliente = spark.read.csv(cliente_path, header=True, inferSchema=True)
produccion_costura_linea_client = spark.read.csv(linea_path, header=True, inferSchema=True)
segundas_prendas = spark.read.csv(segundas_path, header=True, inferSchema=True)

# Convertir columnas numéricas a IntegerType
detalle_produccion = detalle_produccion.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
produccion_costura_cliente = produccion_costura_cliente.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
produccion_costura_linea_client = produccion_costura_linea_client.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
segundas_prendas = segundas_prendas.withColumn("FALLAS_SEGUNDAS", col("FALLAS_SEGUNDAS").cast(IntegerType())) \
                                   .withColumn("INSPECCION_TOTAL", col("INSPECCION_TOTAL").cast(IntegerType()))

# -----------------------------
# Preprocesamiento de FECHA_TERMINO
# -----------------------------
detalle_produccion = detalle_produccion.withColumn(
    "FECHA_TERMINO_TS",
    to_timestamp("FECHA_TERMINO", "dd/MM/yyyy HH:mm:ss")
)

In [17]:
# -----------------------------
# Total prendas por talla
# -----------------------------
detalle_produccion.groupBy("TALLA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TALLA") \
    .write.csv(f"{output_path}total_prendas_por_talla", header=True)

# -----------------------------
# Volumen de ventas por cliente
# -----------------------------
produccion_cliente.groupBy("TCODICLIE") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.csv(f"{output_path}volumen_ventas_por_cliente", header=True)

# -----------------------------
# Fecha ventas
# -----------------------------
produccion_cliente.groupBy("FECHA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("FECHA") \
    .write.csv(f"{output_path}fecha_ventas", header=True)

# -----------------------------
# Tendencias por franja horaria
# -----------------------------
# Convertir FECHA_TERMINO a timestamp y extraer hora
detalle_produccion = detalle_produccion.withColumn(
    "FECHA_TERMINO_TS",
    to_timestamp("FECHA_TERMINO", "dd/MM/yyyy HH:mm:ss")
)
detalle_produccion = detalle_produccion.withColumn("HORA", hour("FECHA_TERMINO_TS"))

tendencias = detalle_produccion.withColumn(
    "FRANJA_HORARIA",
    when((col("HORA") >= 6) & (col("HORA") <= 11), "Mañana")
    .when((col("HORA") >= 12) & (col("HORA") <= 17), "Tarde")
    .when((col("HORA") >= 18) & (col("HORA") <= 23), "Noche")
    .otherwise("Madrugada")
).groupBy("ORDEN_PRODUCCION", "FRANJA_HORARIA") \
 .agg(count("*").alias("TRANSACCIONES"), _sum("PRENDAS").alias("TOTAL_PRENDAS")) \
 .orderBy("ORDEN_PRODUCCION", "FRANJA_HORARIA")

tendencias.write.csv(f"{output_path}tendencias_ventas_por_franja_horaria", header=True)

# -----------------------------
# Productos más vendidos
# -----------------------------
detalle_produccion.groupBy("ESTILO") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("ESTILO") \
    .write.csv(f"{output_path}productos_mas_vendidos", header=True)
# -----------------------------
# Eficiencia operativa
# -----------------------------
segundas_prendas.groupBy("FECHA") \
    .agg(
        round(
            (1 - (_sum("FALLAS_SEGUNDAS") / _sum("INSPECCION_TOTAL"))) * 100, 2
        ).alias("EFICIENCIA_PORCENTUAL")
    ) \
    .orderBy("FECHA") \
    .write.csv(f"{output_path}eficiencia_operativa", header=True)

# -----------------------------
# Índice de ventas por cliente y línea
# -----------------------------
produccion_costura_linea_client.groupBy("TCODICLIE", "LINEA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.csv(f"{output_path}indice_ventas_cliente", header=True)

# -----------------------------
# Predicción de ventas (promedio móvil 7 días)
# -----------------------------
detalle_produccion.groupBy("FECHA_TERMINO_TS", "ESTILO") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .withColumn(
        "PROMEDIO_MOVIL",
        avg("TOTAL_PRENDAS").over(Window.partitionBy("ESTILO").orderBy("FECHA_TERMINO_TS").rowsBetween(-6, 0))
    ) \
    .write.csv(f"{output_path}prediccion_ventas", header=True)

# -----------------------------
# Comportamiento de clientes
# -----------------------------
produccion_costura_cliente.groupBy("TCODICLIE") \
    .agg(count("FECHA").alias("FRECUENCIA_COMPRA"), avg("PRENDAS").alias("PROMEDIO_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.csv(f"{output_path}comportamiento_clientes", header=True)

✅ SCRIPT FINAL CON .mode("overwrite") EN TODAS LAS SALIDAS

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum as _sum, avg, count, round, when, to_timestamp, hour
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

# -----------------------------
# Configuración de Spark
# -----------------------------
spark = SparkSession.builder \
    .appName("Nettalco Big Data Processing") \
    .config("spark.sql.legacy.timeParserPolicy", "LEGACY") \
    .getOrCreate()

# Rutas Cloud Storage
output_path = "gs://nettalco-data-bd_grupo05/curated/"
detalle_path = "gs://nettalco-data-bd_grupo05/detalle-produccion-costura.csv"
cliente_path = "gs://nettalco-data-bd_grupo05/produccion-costura-cliente.csv"
linea_path = "gs://nettalco-data-bd_grupo05/produccion-costura-linea-cliente.csv"
segundas_path = "gs://nettalco-data-bd_grupo05/segundas-prendas.csv"

# Cargar los datos
detalle_produccion = spark.read.csv(detalle_path, header=True, inferSchema=True)
produccion_costura_cliente = spark.read.csv(cliente_path, header=True, inferSchema=True)
produccion_costura_linea_client = spark.read.csv(linea_path, header=True, inferSchema=True)
segundas_prendas = spark.read.csv(segundas_path, header=True, inferSchema=True)

# Convertir columnas numéricas
detalle_produccion = detalle_produccion.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
produccion_costura_cliente = produccion_costura_cliente.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
produccion_costura_linea_client = produccion_costura_linea_client.withColumn("PRENDAS", col("PRENDAS").cast(IntegerType()))
segundas_prendas = segundas_prendas.withColumn("FALLAS_SEGUNDAS", col("FALLAS_SEGUNDAS").cast(IntegerType())) \
                                   .withColumn("INSPECCION_TOTAL", col("INSPECCION_TOTAL").cast(IntegerType()))

# -----------------------------
# Procesar FECHA_TERMINO
# -----------------------------
detalle_produccion = detalle_produccion.withColumn(
    "FECHA_TERMINO_TS",
    to_timestamp("FECHA_TERMINO", "dd/MM/yyyy HH:mm:ss")
)

# -----------------------------
# Total prendas por talla
# -----------------------------
detalle_produccion.groupBy("TALLA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TALLA") \
    .write.mode("overwrite").csv(f"{output_path}total_prendas_por_talla", header=True)

# -----------------------------
# Volumen de ventas por cliente
# -----------------------------
produccion_costura_cliente.groupBy("TCODICLIE") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.mode("overwrite").csv(f"{output_path}volumen_ventas_por_cliente", header=True)

# -----------------------------
# Fecha ventas
# -----------------------------
produccion_costura_cliente.groupBy("FECHA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("FECHA") \
    .write.mode("overwrite").csv(f"{output_path}fecha_ventas", header=True)

# -----------------------------
# Tendencias por franja horaria
# -----------------------------
detalle_produccion = detalle_produccion.withColumn("HORA", hour("FECHA_TERMINO_TS"))

tendencias = detalle_produccion.withColumn(
    "FRANJA_HORARIA",
    when((col("HORA") >= 6) & (col("HORA") <= 11), "Mañana")
    .when((col("HORA") >= 12) & (col("HORA") <= 17), "Tarde")
    .when((col("HORA") >= 18) & (col("HORA") <= 23), "Noche")
    .otherwise("Madrugada")
).groupBy("ORDEN_PRODUCCION", "FRANJA_HORARIA") \
 .agg(count("*").alias("TRANSACCIONES"), _sum("PRENDAS").alias("TOTAL_PRENDAS")) \
 .orderBy("ORDEN_PRODUCCION", "FRANJA_HORARIA")

tendencias.write.mode("overwrite").csv(f"{output_path}tendencias_ventas_por_franja_horaria", header=True)

# -----------------------------
# Productos más vendidos
# -----------------------------
detalle_produccion.groupBy("ESTILO") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("ESTILO") \
    .write.mode("overwrite").csv(f"{output_path}productos_mas_vendidos", header=True)

# -----------------------------
# Eficiencia operativa
# -----------------------------
segundas_prendas.groupBy("FECHA") \
    .agg(
        round(
            (1 - (_sum("FALLAS_SEGUNDAS") / _sum("INSPECCION_TOTAL"))) * 100, 2
        ).alias("EFICIENCIA_PORCENTUAL")
    ) \
    .orderBy("FECHA") \
    .write.mode("overwrite").csv(f"{output_path}eficiencia_operativa", header=True)

# -----------------------------
# Índice de ventas por cliente y línea
# -----------------------------
produccion_costura_linea_client.groupBy("TCODICLIE", "LINEA") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.mode("overwrite").csv(f"{output_path}indice_ventas_cliente", header=True)

# -----------------------------
# Predicción de ventas (promedio móvil 7 días)
# -----------------------------
detalle_produccion.groupBy("FECHA_TERMINO_TS", "ESTILO") \
    .agg(_sum("PRENDAS").alias("TOTAL_PRENDAS")) \
    .withColumn(
        "PROMEDIO_MOVIL",
        avg("TOTAL_PRENDAS").over(Window.partitionBy("ESTILO").orderBy("FECHA_TERMINO_TS").rowsBetween(-6, 0))
    ) \
    .write.mode("overwrite").csv(f"{output_path}prediccion_ventas", header=True)

# -----------------------------
# Comportamiento de clientes
# -----------------------------
produccion_costura_cliente.groupBy("TCODICLIE") \
    .agg(count("FECHA").alias("FRECUENCIA_COMPRA"), avg("PRENDAS").alias("PROMEDIO_PRENDAS")) \
    .orderBy("TCODICLIE") \
    .write.mode("overwrite").csv(f"{output_path}comportamiento_clientes", header=True)